In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

## IMPORTACIÓN

In [71]:
# Importación de datos
d = pd.read_csv('Datos_Medellín/combined.csv') 
d.head(2)

,Fecha_Hora,codigoSerial,pm25,calidad_pm25,pm10,calidad_pm10,pm1,calidad_pm1,no,calidad_no,...,p_ssr,calidad_p_ssr,pliquida_ssr,calidad_pliquida_ssr,rglobal_ssr,calidad_rglobal_ssr,taire10_ssr,calidad_taire10_ssr,vviento_ssr,calidad_vviento_ssr
0,2020-02-01 00:00:00,100,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,2020-02-01 01:00:00,100,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [72]:
datos = d

In [73]:
np.shape(datos)

(1683595, 36)

## LIMPIAR

In [74]:
conservar = ['Fecha_Hora', 'codigoSerial', 'pm25','pm10']
datos = datos[conservar]

In [75]:
pos_del = np.where(datos['codigoSerial'] == 'codigoSerial')[0]
datos = datos.drop(pos_del)

In [76]:
np.shape(datos)

(1681296, 4)

In [77]:
datos['Fecha_Hora'] = pd.to_datetime(datos['Fecha_Hora'])

In [78]:
datos = datos.loc[pd.DatetimeIndex(datos['Fecha_Hora']).year > 2018]

In [79]:
datos.iloc[:,1:] = datos.iloc[:,1:].astype(float)

In [80]:
"""
datos['pm25'].loc[ 
    (datos['pm25']< 0) | 
    (datos['pm25']> 200)] = datos['pm25'].loc[ (datos['pm25'] > 0) & 
                                              (datos['pm25'] < 200)].mean()

datos['pm10'].loc[ (
    datos['pm10']< 0) | 
    (datos['pm10'] > 200)] = datos['pm10'].loc[ (datos['pm10'] > 0) & 
                                              (datos['pm10'] < 200)].mean()
                                              
"""

"\ndatos['pm25'].loc[ \n    (datos['pm25']< 0) | \n    (datos['pm25']> 200)] = datos['pm25'].loc[ (datos['pm25'] > 0) & \n                                              (datos['pm25'] < 200)].mean()\n\ndatos['pm10'].loc[ (\n    datos['pm10']< 0) | \n    (datos['pm10'] > 200)] = datos['pm10'].loc[ (datos['pm10'] > 0) & \n                                              (datos['pm10'] < 200)].mean()\n                                              \n"

In [81]:
datos['pm25'].loc[ (datos['pm25']<= 0) | (datos['pm25']> 200)] = None

datos['pm10'].loc[ (datos['pm10']<= 0) | (datos['pm10'] > 200)] = None

/Users/PutosHackers/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
np.mean(datos['pm10'])

42.78264053427989

## DATOS GRANDES ERRONEOS

In [83]:
datos['Semana'] = datos['Fecha_Hora'].dt.dayofweek

## SEMANAL

In [84]:
import calendar
days = list(calendar.day_abbr)
months = list(calendar.month_abbr)[1:]

In [85]:
#top n estaciones pm25

def get_top_estaciones(valor, n):
    
    #promedio total (mensual)
    d_total_mensual = datos.groupby(by= [datos['codigoSerial']]).mean()
    d_total_mensual = d_total_mensual.reset_index()
    d_total_mensual['Total'] = d_total_mensual['pm25'] + d_total_mensual['pm10']
    
    print(np.shape(d_total_mensual))
    
    #datos organizados
    total = d_total_mensual.sort_values(by= valor, ascending= False).head(n)
    
    #index del top
    max_est = np.array(list(set(total['codigoSerial'].values)))
    return np.sort(max_est)
    

max_est_25 = get_top_estaciones('pm25', 3)
max_est_10 = get_top_estaciones('pm10', 3)

(32, 5)
(32, 5)


In [86]:
def get_semanal(datos):
    d_semanal = datos.groupby(by= [datos['codigoSerial'], datos['Semana']]).mean()
    d_semanal = d_semanal.reset_index()
    return d_semanal

d_semanal = get_semanal(datos)

In [87]:
def get_estaciones(datos):
    return np.array(list(set(datos['codigoSerial'].values)))

In [88]:
colors = [
    '#fcbc45',
    '#5e1e4e',
    '#2b5259',
    '#2b2b2b',
    '#417b85',
    '#91bcab'
]

In [89]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def fig_bar(max_est_1, max_est_2, datos):
    
    d_semanal = get_semanal(datos)
    
    
    #figura
    fig = make_subplots(rows=2, 
                        cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing = 0.03,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                       )

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['Semana']+1, y=datos_temp['pm25'],
                             name='Estación ' + str(int(max_est_1[i])),
                             legendgroup = 'Estación ' + str(int(max_est_1[i])),
                             opacity= op,
                             marker_color = colors[i]
                            ),
                      row=1, col=1)



    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Bar(x=datos_temp['Semana']+1, y=datos_temp['pm10'],
                             name='Estación ' + str(int(max_est_2[i])),
                             legendgroup = 'Estación ' + str(int(max_est_2[i])),
                             opacity= op,
                             marker_color = color,
                             showlegend=showlegend
                            ),
                     row=2, col=1)   
        
    #Total
    
    d_temp = datos.groupby(by = datos['Semana']).mean()[['pm25', 'pm10']]
    d_temp = d_temp.reset_index()

    fig.add_trace(go.Bar(x=d_temp['Semana']+1, y=d_temp['pm25'],
                         name='Promedio total ',
                         opacity= op,
                         legendgroup = 'Promedio total ',
                         marker_color = colors[i+1]
                                ),
                  row=1, col=1)    

    fig.add_trace(go.Bar(x=d_temp['Semana']+1, y=d_temp['pm10'],
                         name='Promedio total ',
                         legendgroup = 'Promedio total ',
                         opacity= op,
                         marker_color = colors[i+1],
                         showlegend=False),
                  row=2, col=1)   
    
    # Porcentaje diario
    
    d_temp[['pm25','pm10']] = d_temp[['pm25','pm10']]/np.sum(d_temp[['pm25','pm10']])
    
    fig.add_trace(go.Scatter(x=d_temp['Semana']+1, y=d_temp['pm25'],
                             name = 'Porcentaje dia de la semana',
                             legendgroup = 'Porcentaje dia de la semana',
                             mode = 'lines+markers',
                             showlegend = True,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=1, col=1, secondary_y=True,)
    
    fig.add_trace(go.Scatter(x=d_temp['Semana']+1, y=d_temp['pm10'],
                             name = 'Porcentaje dia de la semana',
                             legendgroup = 'Porcentaje dia de la semana',
                             mode = 'lines+markers',
                             showlegend = False,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=2, col=1, secondary_y=True,)
       
        

    
    #Figure layout
    
    fig.update_yaxes(title_text= 'PM 2.5 (\u03BCg/m\u00b3)',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'PM 10 (\u03BCg/m\u00b3)', row=2, col=1, secondary_y=False)
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)

    
    fig.update_xaxes(title_text='Día',
                     tickmode = 'array',
                     tickvals = np.arange(1,8),
                     ticktext= days)
    
    
    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Promedio diario por estación',
                      title_font_size=25, 
                      font_size=17,
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.0,
                                    xanchor="right",
                                    x=0.97
                                )
                     )
    fig.show()

    #Write HTML file
    fig.write_html("Datos_Medellín/web_comparativo_diario.html")

d_temp = fig_bar(max_est_25, max_est_10, datos)

# DIARIO

In [90]:
def get_hora(datos):
    d_hora = datos.groupby(by  = [datos['codigoSerial'], pd.DatetimeIndex(datos['Fecha_Hora']).hour]).mean()
    d_hora = d_hora.reset_index()
    return d_hora

In [91]:
hours = [(n, "%d %s" % (n % 12 or 12, ["AM", "PM"][n > 11])) for n in range(24)]
hours = np.array(hours)
hours[:,0] = hours[:,0].astype('int') 

c_map = [[20,'rgb(94,3078)']]

In [92]:
import plotly.io as pio
list(pio.templates)

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [93]:
def fig_bar_hora(max_est_1, max_est_2, datos):
    
    d_semanal = get_hora(datos)
    
    
    #figura
    fig = go.Figure()

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['pm25'], y=datos_temp['Fecha_Hora'],
                             name='Estación ' + str(int(max_est_1[i])),
                             legendgroup = 'Estación ' + str(int(max_est_1[i])),
                            showlegend = False,
                             opacity= op,
                             orientation='h',
                             marker=dict(color=datos_temp['pm25'], 
                                        colorbar=dict(title="pm 2.5 <br> (\u03BCg/m\u00b3)",
                                                     bordercolor= 'white'),
                                         colorscale=['rgb(145,188,171)','rgb(65,123,133)',
                                                     'rgb(94,30,78)', 'rgb(252,188,69)', 'indianred']
                                        )
                            )
                     )
        break

        
        
    fig.add_trace(go.Scatter(x=[10,10], y=[0,23],
                             name = 'Límite OMS',
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='white', width=1.5)))

    #Figure layout
    
    fig.update_xaxes( showticklabels=False)
    
    

    
    fig.update_yaxes(tickmode = 'array',
                    tickvals = hours[1::2,0].astype('int'),
                    ticktext = hours[1::2,1], title= 'Hora')
    
    
    dim = 1200

    fig.update_layout(height= 2436/2, width=1125/2,
                      title_font_size=25, 
                      font_size=17,
                      template="plotly_dark",
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.0,
                                    xanchor="right",
                                    x=0.75
                                )
                     )
    fig.show()

    #Write HTML file
    fig.write_html("Datos_Medellín/web_comparativo_hora.html")

d_temp = fig_bar_hora(max_est_25, max_est_10, datos)

## MENSUAL

In [94]:
def get_mensual(datos):
    d_mensual = datos.groupby(by= [datos['codigoSerial'], pd.DatetimeIndex(datos['Fecha_Hora']).month]).mean()
    d_mensual = d_mensual.reset_index()
    return d_mensual

d_mensual = get_mensual(datos)

In [95]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def fig_bar(max_est_1, max_est_2, datos):
    
    d_semanal = get_mensual(datos)
    
    
    #figura
    fig = make_subplots(rows=2, 
                        cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing = 0.03,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                       )

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm25'],
                             name='Estación ' + str(int(max_est_1[i])),
                             legendgroup = 'Estación ' + str(int(max_est_1[i])),
                             opacity= op,
                             marker_color = colors[i]
                            ),
                      row=1, col=1)



    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm10'],
                             name='Estación ' + str(int(max_est_2[i])),
                             legendgroup = 'Estación ' + str(int(max_est_2[i])),
                             opacity= op,
                             marker_color = color,
                             showlegend=showlegend
                            ),
                     row=2, col=1)   
        
    #Total
    
    d_temp = d_mensual.groupby(by = d_mensual['Fecha_Hora']).mean()[['pm25', 'pm10']]
    d_temp = d_temp.reset_index()

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                         name='Promedio total ',
                         opacity= op,
                         legendgroup = 'Promedio total ',
                         marker_color = colors[i+1]
                                ),
                  row=1, col=1)    

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                         name='Promedio total ',
                         legendgroup = 'Promedio total ',
                         opacity= op,
                         marker_color = colors[i+1],
                         showlegend=False),
                  row=2, col=1)   
    
    # Porcentaje diario
    
    
    d_temp[['pm25','pm10']] = d_temp[['pm25','pm10']]/np.sum(d_temp[['pm25','pm10']])
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = True,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=1, col=1, secondary_y=True,)
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = False,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=2, col=1, secondary_y=True,)
    
    
    
    #Figure layout
    
    fig.update_yaxes(title_text= 'PM 2.5 (\u03BCg/m\u00b3)',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'PM 10 (\u03BCg/m\u00b3)', row=2, col=1, secondary_y=False)
    
    fig.update_xaxes(title_text='Mes')
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)
    

    fig.update_xaxes(tickmode = 'array',
                 tickvals = np.arange(1,13),
                 ticktext= months)

    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Promedio mensual por estación',
                      title_font_size=25, 
                      font_size=18,
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.01,
                                    xanchor="right",
                                    x=0.97
                                )
                     )
    fig.show()

    #Write HTML file
    fig.write_html("Datos_Medellín/web_comparativo_mensual.html")

d_temp = fig_bar(max_est_25, max_est_10, datos)

## CONTINUO

In [96]:
def get_datos_anual(datos):
    d_anual= datos
    
    #retrieve año mes y dia
    d_anual['Año'] =  pd.DatetimeIndex(d_anual['Fecha_Hora']).year
    d_anual['Mes'] =  pd.DatetimeIndex(d_anual['Fecha_Hora']).month
    d_anual['Dia'] =  pd.DatetimeIndex(d_anual['Fecha_Hora']).day
    
    #group by without adding index
    d_anual = d_anual.groupby(by= [d_anual['codigoSerial'], 
                   d_anual['Año'],
                   d_anual['Mes'],
                   d_anual['Dia'],
                  ],
             as_index = False).mean()
    
    #change type to str
    d_anual[['Año','Mes','Dia']] = d_anual[['Año','Mes','Dia']].astype(str)

    
    #add cero to one digit dates
    uno_len_d = np.where(d_anual['Dia'].str.len() == 1)
    d_anual['Dia'].iloc[uno_len_d] = '0' + d_anual['Dia'].iloc[uno_len_d]

    uno_len_m = np.where(d_anual['Mes'].str.len() == 1)
    d_anual['Mes'].iloc[uno_len_m] = '0' + d_anual['Mes'].iloc[uno_len_m]

    #add column in datetime format
    d_anual['Fecha_Hora'] = d_anual['Año'] + d_anual['Mes'] + d_anual['Dia']
    
    #erase innecesary columns    
    d_anual = d_anual.drop(['Semana','Mes','Dia'],axis =1)
    
    
    #column format to date
    d_anual['Fecha_Hora'] = pd.to_datetime(d_anual['Fecha_Hora'], format='%Y%m%d')
    
    return d_anual

d_anual = get_datos_anual(datos)

In [97]:
np.min(datos['Fecha_Hora']), np.max(datos['Fecha_Hora'])

(Timestamp('2019-01-01 00:00:00'), Timestamp('2021-01-31 23:00:00'))

In [98]:
def scatter_datos(max_est_1, max_est_2, datos):
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

    op = 0.75

    s = 10

    w = 4

    span = 30
    #ADD TRACES 
    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + str(int(max_est_1[i])),
                                 legendgroup = 'Estación ' + str(int(max_est_1[i])),
                                 marker=dict(
                                     size=s,
                                     color = colors[i],)
                                ),
                      row=1, col=1)

    ##2
        
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'], y = datos_temp['pm10'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + str(int(max_est_2[i])),
                                 legendgroup = 'Estación ' + str(int(max_est_2[i])),
                                 showlegend = showlegend,
                                 marker=dict(
                                     size=s,
                                     color = color,)
                                ),
                      row=2, col=1)

    
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=span,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=span,adjust=False).mean()



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Media Móvil',
                             legendgroup = 'Media Móvil',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=1)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Otras',
                             legendgroup = 'Media Móvil',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=1)

    
    #linea horizontal    
    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=1, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=1, secondary_y=False,)
    
    
    # FIGURE LAYOUT
    
    
    dim = 1200
    
    fig.update_layout(height=9*dim/16, width=dim, title= 'Registros diarios por estación',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.01,
                          xanchor="right", 
                          x = 1.)                 
                 )
    
    fig.update_yaxes(title= 'PM 2.5 (\u03BCg/m\u00b3)',  row=1, col=1)
    fig.update_yaxes(title= 'PM 10 (\u03BCg/m\u00b3)', row=2, col=1)
    
    fig.update_xaxes(title_text='Año')
    
    fig.update_xaxes(range = ['2018-12-20','2021-02-15'])
    
    
    fig.show()
    
    
    #Write HTML file
    fig.write_html("Datos_Medellín/web_comparativo_estacion.html")
        
scatter_datos(max_est_25, max_est_10, d_anual)

# pm2.5 = 10 
#pm 10 = 

## PORCENTUAL

In [ ]:
def medias_moviles(max_est_1, max_est_2, datos):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                   )
    
    op = 0.85

    s = 10

    w = 4
    
    # TRACES
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=30,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=30,adjust=False).mean()


    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=1)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=1)

    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):
    
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=40,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=40,adjust=False).mean()

        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 opacity= op,
                                 name = 'Estación ' + str(int(max_est_1[i])),
                                 legendgroup = 'Estación ' + str(int(max_est_1[i])),
                                 line = dict(color=colors[i], width=w)),
                      row=1, col=1)

    ##2
    
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=40,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=40,adjust=False).mean()
        
        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm10'],                     
                                 opacity= op,
                                 name = 'Estación ' + str(int(max_est_2[i])),
                                 legendgroup = 'Estación ' + str(int(max_est_2[i])),
                                 showlegend = showlegend,
                                 line = dict(color=color, width=w)),
                      row=2, col=1)

    
    
    #linea horizontal    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=1, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=1, secondary_y=False,)
    
    
    
    # UPDATE LAYOUT
    fig.update_yaxes(title_text= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1, secondary_y=False)
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)


    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Media móvil por estación (2019-2021)',
                      title_font_size=20, 
                      font_size=12,
                      legend=dict(orientation="h",
                                  yanchor="bottom",
                                  y=1.0,
                                  xanchor="right",
                                  x=0.75
                                )
                     )
    
    fig.show()
    fig.write_html("Datos_2000_2020/comparativo_media_móvil.html")

medias_moviles(max_est_25, max_est_10, d_anual)

## DIARIO

In [27]:
np.shape(d_diario)

NameError: name 'd_diario' is not defined

In [ ]:
d_diario['Mes'] = d_diario.index.month
d_diario['Dia'] = d_diario.index.day

In [ ]:
d_mean = d_diario.groupby(by=[d_diario.index.month, d_diario.index.day]).mean()

In [ ]:
d_mean = d_mean.rename(columns={'calidad_pm25':'mean_pm25'})

In [ ]:
new_df = pd.merge(d_diario, d_mean,  how='right', on=['Mes','Dia'])

In [ ]:
new_df.index = d_diario.index

In [ ]:
new_df = new_df.between_time('00:00:00','5:00:00')
#new_df = new_df.drop('mean_pm25', axis=1)

In [ ]:
X = np.array(new_df.iloc[:,:-1].T)
Y = np.array(new_df.iloc[:,-1])

In [ ]:
X

In [ ]:
Y = Y/np.max(Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.T, Y, test_size=0.33, random_state=42)

In [ ]:
from tensorflow import keras

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)])
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
         # Your Code
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.90):
                print("/nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                        tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                        tf.keras.layers.Dense(128, activation= tf.nn.relu),
                                        tf.keras.layers.Dropout(0.2),
                                        tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)
                                       ])


    model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['acc'])
        


    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(X_train, y_train,
        steps_per_epoch=8,  
        epochs=10,
        verbose=1,
        callbacks = [callbacks]
    )
    
    # model fitting
    return history.history['acc'][-1]

In [ ]:
train_happy_sad_model()